In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc
import os

In [3]:
# Load test_sup and find index
load_path = '/home/kai/data/kaggle/talkingdata/downsampling/'                     
print('sup read done')

sup read done
slicing without correction done


In [5]:
load_path = '/home/kai/data/kaggle/talkingdata/downsampling/'
file_format = '{}.ftr'
# train = pd.read_feather(poad_path + 'train_cleaned.ftr')
train = pd.read_feather(load_path + '0.ftr')
    
print('train(downsampled) done')
val = pd.read_feather(load_path + 'val.ftr')
print('val done')

train(downsampled) done
val done


In [10]:
import random as rn
import os

target = 'is_attributed'
os.environ['PYTHONHASHSEED'] = '0'
seed = 2018
np.random.seed(seed)
rn.seed(seed)
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_boost_round': 2000,
        'learning_rate': 0.05,
        'num_leaves': 70,
        'num_threads': 16, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': int(np.log2(74)+1.2), # no limit. This is used to deal with over-fitting when #data is small.
        'min_child_samples': 20,
        'min_child_weight': 5,
        'min_split_gain': 0,
        'reg_alpha': 0,
        'reg_lambda': 0,
        'subsample': 1.0,
        'subsample_for_bin': 200000,
        'subsample_freq': 1,
        'bagging_seed': seed,
        'early_stopping_round':100,
        'verbose': 0,
        'scale_pos_weight': 5,
        'metric' : [ 'auc'],
        'bagging_seed': seed,
        'feature_fraction_seed': seed,
        'feature_fraction': 0.7,
        'drop_seed': seed,
        'data_random_seed': seed,
        'max_bin': 305,
        'colsample_bytree': 309
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour', 'range_count_ip_day_hour']
feature_cols = list(train.columns.values)
remove_list = [
    'count_app_channel',
    'count_ip_day',
    'count_app_day_hourminute',
    'unique_ratio_day_ip_machine',
    'unique_ratio_day_ip_device',
    'unique_ratio_day_ip_app',
    'unique_ratio_machine_channel',
    'unique_ratio_day_ip_os',
    'unique_ratio_machine_app',
    'unique_ratio_machine_ip',
    'cumcount_ip_app_device_os_day_hour',
    'range_count_ip_channel_dayhourminute',
    'range_count_ip_day',
    'range_count_ip_device_os_dayhourminute',
    'range_count_ip_dayhourminute',
    'range_count_ip_dayhourminute10',
    'range_count_app_os_channel_dayhourminute',
    'com_ip','index', 'click_id', 'ip','click_time', 'is_test', 'is_attributed', 'day',
    'minute', 'minute10', 'hourminute', 'hourminute10', 'dayhourminute', 'dayhourminute10', 'machine', 'click_timestamp',
    'filter_time_to_n_next_click_2_day_ip_app_device_os',
    'filter_time_to_n_next_click_1_day_ip_app_device_os'
]
feature_cols = [x for x in feature_cols if x not in remove_list]
print(len(feature_cols))
# print(feature_cols)

83


In [11]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model


# Train model and get prediction

In [12]:
model = train_lightgbm(train, val, feature_cols, categorical_col, params)
print('prediction done')

start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.981003
[20]	valid_0's auc: 0.981718
[30]	valid_0's auc: 0.982457
[40]	valid_0's auc: 0.98358
[50]	valid_0's auc: 0.984147
[60]	valid_0's auc: 0.984627
[70]	valid_0's auc: 0.984823
[80]	valid_0's auc: 0.984884
[90]	valid_0's auc: 0.984894
[100]	valid_0's auc: 0.984987
[110]	valid_0's auc: 0.985067
[120]	valid_0's auc: 0.985049
[130]	valid_0's auc: 0.985038
[140]	valid_0's auc: 0.985108
[150]	valid_0's auc: 0.985162
[160]	valid_0's auc: 0.985194
[170]	valid_0's auc: 0.985193
[180]	valid_0's auc: 0.985175
[190]	valid_0's auc: 0.985119
[200]	valid_0's auc: 0.985136
[210]	valid_0's auc: 0.98513
[220]	valid_0's auc: 0.985149
[230]	valid_0's auc: 0.985147
[240]	valid_0's auc: 0.985154
[250]	valid_0's auc: 0.985163
[260]	valid_0's auc: 0.985132
Early stopping, best iteration is:
[169]	valid_0's auc: 0.985229
prediction done


In [13]:
# test_supplement = pd.read_feather(load_path + 'test_cleaned.ftr')
# test_index = np.load('/home/kai/data/kaggle/talkingdata/data/supplement2testInexFinal.npy')
# test = test_supplement.iloc[test_index]
# print(test.shape)
# pred = model.predict(test[feature_cols])
# print('done')

# Submit result

In [ ]:
sub_dict = {}
df_sub = pd.DataFrame()
test_clickid = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv', usecols=['click_id'])
print('done')
df_sub['click_id'] = test_clickid['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/downsampling/result/b1.csv.gz', compression='gzip', index=False)